In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from model import GTN
import pdb
import pickle
import argparse
from utils import f1_score

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='DBLP',
                    help='Dataset')
parser.add_argument('--epoch', type=int, default=40,
                    help='Training Epochs')
parser.add_argument('--node_dim', type=int, default=64,
                    help='Node dimension')
parser.add_argument('--num_channels', type=int, default=2,
                    help='number of channels')
parser.add_argument('--lr', type=float, default=0.005,
                    help='learning rate')
parser.add_argument('--weight_decay', type=float, default=0.001,
                    help='l2 reg')
parser.add_argument('--num_layers', type=int, default=2,
                    help='number of layer')
parser.add_argument('--norm', type=str, default='true',
                    help='normalization')
parser.add_argument('--adaptive_lr', type=str, default='false',
                    help='adaptive learning rate')

#args = parser.parse_args()                                               # pychram 中使用
args = parser.parse_known_args()[0]                                       # jupyter 中使用
print(args)

Namespace(dataset='DBLP', epoch=40, node_dim=64, num_channels=2, lr=0.005, weight_decay=0.001, num_layers=2, norm='true', adaptive_lr='false')


In [5]:
epochs = args.epoch
node_dim = args.node_dim
num_channels = args.num_channels
lr = args.lr
weight_decay = args.weight_decay
num_layers = args.num_layers
norm = args.norm
adaptive_lr = args.adaptive_lr

In [14]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
with open('../data/'+args.dataset+'/node_features.pkl','rb') as f:
    node_features = pickle.load(f)
with open('../data/'+args.dataset+'/edges.pkl','rb') as f:
    edges = pickle.load(f)
with open('../data/'+args.dataset+'/labels.pkl','rb') as f:
    labels = pickle.load(f)
num_nodes = edges[0].shape[0]

In [8]:
for i,edge in enumerate(edges):
    if i ==0:
        A = torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)
    else:
        A = torch.cat([A,torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)], dim=-1)
A = torch.cat([A,torch.eye(num_nodes).type(torch.FloatTensor).unsqueeze(-1)], dim=-1)

In [9]:
A.shape

torch.Size([18405, 18405, 5])

In [10]:
node_features = torch.from_numpy(node_features).type(torch.FloatTensor)
train_node = torch.from_numpy(np.array(labels[0])[:,0]).type(torch.LongTensor)
train_target = torch.from_numpy(np.array(labels[0])[:,1]).type(torch.LongTensor)
valid_node = torch.from_numpy(np.array(labels[1])[:,0]).type(torch.LongTensor)
valid_target = torch.from_numpy(np.array(labels[1])[:,1]).type(torch.LongTensor)
test_node = torch.from_numpy(np.array(labels[2])[:,0]).type(torch.LongTensor)
test_target = torch.from_numpy(np.array(labels[2])[:,1]).type(torch.LongTensor)

In [11]:
print(node_features.shape)
print(train_node.shape)
print(train_target.shape)
print(valid_node.shape)
print(valid_target.shape)
print(test_node.shape)
print(test_target.shape)
print(A.shape)

torch.Size([18405, 334])
torch.Size([800])
torch.Size([800])
torch.Size([400])
torch.Size([400])
torch.Size([2857])
torch.Size([2857])
torch.Size([18405, 18405, 5])


In [12]:
num_classes = torch.max(train_target).item()+1
final_f1 = 0
num_classes

4

In [13]:
for l in range(1):
    model = GTN(num_edge=A.shape[-1],
                        num_channels=num_channels,
                        w_in = node_features.shape[1],
                        w_out = node_dim,
                        num_class=num_classes,
                        num_layers=num_layers,
                        norm=norm)
    if adaptive_lr == 'false':
        optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
    else:
        optimizer = torch.optim.Adam([{'params':model.weight},
                                    {'params':model.linear1.parameters()},
                                    {'params':model.linear2.parameters()},
                                    {"params":model.layers.parameters(), "lr":0.5}
                                    ], lr=0.005, weight_decay=0.001)
    loss = nn.CrossEntropyLoss()
    # Train & Valid & Test
    best_val_loss = 10000
    best_test_loss = 10000
    best_train_loss = 10000
    best_train_f1 = 0
    best_val_f1 = 0
    best_test_f1 = 0
    
    # for i in range(epochs):
    for i in range(1):
        for param_group in optimizer.param_groups:
            if param_group['lr'] > 0.005:
                param_group['lr'] = param_group['lr'] * 0.9
        print('Epoch:  ',i+1)
        model.zero_grad()
        model.train()
        loss,y_train,Ws = model(A, node_features, train_node, train_target)
        train_f1 = torch.mean(f1_score(torch.argmax(y_train.detach(),dim=1), train_target, num_classes=num_classes)).cpu().numpy()
        print('Train - Loss: {}, Macro_F1: {}'.format(loss.detach().cpu().numpy(), train_f1))
        loss.backward()
        optimizer.step()
        model.eval()

        # Valid
        with torch.no_grad():
            val_loss, y_valid,_ = model.forward(A, node_features, valid_node, valid_target)
            val_f1 = torch.mean(f1_score(torch.argmax(y_valid,dim=1), valid_target, num_classes=num_classes)).cpu().numpy()
            print('Valid - Loss: {}, Macro_F1: {}'.format(val_loss.detach().cpu().numpy(), val_f1))
            test_loss, y_test,W = model.forward(A, node_features, test_node, test_target)
            test_f1 = torch.mean(f1_score(torch.argmax(y_test,dim=1), test_target, num_classes=num_classes)).cpu().numpy()
            print('Test - Loss: {}, Macro_F1: {}\n'.format(test_loss.detach().cpu().numpy(), test_f1))
        
        if val_f1 > best_val_f1:
            best_val_loss = val_loss.detach().cpu().numpy()
            best_test_loss = test_loss.detach().cpu().numpy()
            best_train_loss = loss.detach().cpu().numpy()
            best_train_f1 = train_f1
            best_val_f1 = val_f1
            best_test_f1 = test_f1 
            
    print('---------------Best Results--------------------')
    print('Train - Loss: {}, Macro_F1: {}'.format(best_train_loss, best_train_f1))
    print('Valid - Loss: {}, Macro_F1: {}'.format(best_val_loss, best_val_f1))
    print('Test - Loss: {}, Macro_F1: {}'.format(best_test_loss, best_test_f1))
    final_f1 += best_test_f1

Epoch:   1
Train - Loss: 1.390804409980774, Macro_F1: 0.11158082634210587
Valid - Loss: 1.3697787523269653, Macro_F1: 0.2811485528945923
Test - Loss: 1.3840534687042236, Macro_F1: 0.24715301394462585

---------------Best Results--------------------
Train - Loss: 1.390804409980774, Macro_F1: 0.11158082634210587
Valid - Loss: 1.3697787523269653, Macro_F1: 0.2811485528945923
Test - Loss: 1.3840534687042236, Macro_F1: 0.24715301394462585
